In [1]:
import pandas as pd
root = 'Dota_ML'
df_match = pd.read_csv('match.csv')
df_player_time = pd.read_csv('player_time.csv')
# link to datasets: https://www.kaggle.com/devinanzelmo/dota-2-matches

Research question: Can we accurately predict the outcome of dota matches based off of team networth, xp gain, and lasthits alone?

In [2]:
df_match.head()

,match_id,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
0,0,1446750112,2375,1982,4,3,63,1,22,True,0,1,155
1,1,1446753078,2582,0,1846,63,0,221,22,False,0,2,154
2,2,1446764586,2716,256,1972,63,48,190,22,False,0,0,132
3,3,1446765723,3085,4,1924,51,3,40,22,False,0,0,191
4,4,1446796385,1887,2047,0,0,63,58,22,True,0,0,156


In [3]:
df_match.columns

Index(['match_id', 'start_time', 'duration', 'tower_status_radiant',
       'tower_status_dire', 'barracks_status_dire', 'barracks_status_radiant',
       'first_blood_time', 'game_mode', 'radiant_win', 'negative_votes',
       'positive_votes', 'cluster'],
      dtype='object')

In [4]:
df_player_time.head()

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589


In [5]:
df_player_time.columns

Index(['match_id', 'times', 'gold_t_0', 'lh_t_0', 'xp_t_0', 'gold_t_1',
       'lh_t_1', 'xp_t_1', 'gold_t_2', 'lh_t_2', 'xp_t_2', 'gold_t_3',
       'lh_t_3', 'xp_t_3', 'gold_t_4', 'lh_t_4', 'xp_t_4', 'gold_t_128',
       'lh_t_128', 'xp_t_128', 'gold_t_129', 'lh_t_129', 'xp_t_129',
       'gold_t_130', 'lh_t_130', 'xp_t_130', 'gold_t_131', 'lh_t_131',
       'xp_t_131', 'gold_t_132', 'lh_t_132', 'xp_t_132'],
      dtype='object')

In [6]:
60*15

900

In [7]:
# Make sure game_mode is all pick ranked only. 
df_match = df_match.loc[df_match['game_mode']==22]

# remove redundnt features

df_match = df_match.drop(['negative_votes','positive_votes','cluster'], axis=1)

# Remove instances of match time = 0 
df_player_time = df_player_time[df_player_time['times']!=0]

In [8]:
# aggregate variable data inputs
def create_feature(cols_list, name):
    df_player_time[name] = df_player_time[cols_list].sum(axis=1)

    # sum of radiant gold
cols_to_sum = ['gold_t_0', 'gold_t_1', 'gold_t_2', 'gold_t_3', 'gold_t_4']
create_feature(cols_to_sum, 'radiant_gold')

    # sum of radiant last hits
cols_to_sum = ['lh_t_0', 'lh_t_1', 'lh_t_2', 'lh_t_3', 'lh_t_4']
create_feature(cols_to_sum, 'radiant_lh')
    # sum of radiant xp
cols_to_sum = ['xp_t_0', 'xp_t_1', 'xp_t_2', 'xp_t_3', 'xp_t_4']
create_feature(cols_to_sum, 'radiant_xp')
##
##
    # sum of dire gold
cols_to_sum = ['gold_t_128', 'gold_t_129', 'gold_t_130', 'gold_t_131', 'gold_t_132']
create_feature(cols_to_sum, 'dire_gold')

    # sum of dire last hits
cols_to_sum = ['lh_t_128', 'lh_t_129', 'lh_t_130', 'lh_t_131', 'lh_t_132']
create_feature(cols_to_sum, 'dire_lh')

    # sum of dire xp
cols_to_sum = ['xp_t_128', 'xp_t_129', 'xp_t_130', 'xp_t_131', 'xp_t_132']
create_feature(cols_to_sum, 'dire_xp')

In [9]:
df_player_time.head()

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_131,gold_t_132,lh_t_132,xp_t_132,radiant_gold,radiant_lh,radiant_xp,dire_gold,dire_lh,dire_xp
1,0,60,409,0,63,142,1,186,168,0,...,77,613,1,125,1113,2,692,1370,10,692
2,0,120,546,0,283,622,4,645,330,0,...,210,815,5,323,2471,10,2376,2726,26,2318
3,0,180,683,1,314,927,9,1202,430,0,...,210,1290,8,527,3490,23,3634,4057,40,3543
4,0,240,956,1,485,1264,11,1583,530,0,...,210,1431,9,589,4950,33,5285,5500,53,5007
5,0,300,1056,1,649,1451,13,1810,630,0,...,241,2110,17,918,5947,46,6484,6984,71,6574


In [10]:
# Calculate team advantages

    # radiant_adv networth
df_player_time['radiant_adv_nw'] = df_player_time.apply(lambda x: x['radiant_gold'] - x['dire_gold'], axis=1)

    # radiant_adv xp
df_player_time['radiant_adv_xp'] = df_player_time.apply(lambda x: x['radiant_xp'] - x['dire_xp'], axis=1)

    # radiant_adv lasthits 
df_player_time['radiant_adv_lh'] = df_player_time.apply(lambda x: x['radiant_lh'] - x['dire_lh'], axis=1)

    # dire_adv networth
df_player_time['dire_adv_nw'] = df_player_time.apply(lambda x: x['dire_gold'] - x['radiant_gold'], axis=1)

    # dire_adv xp
df_player_time['dire_adv_xp'] = df_player_time.apply(lambda x: x['dire_xp'] - x['radiant_gold'], axis=1)

    # dire_adv lasthits 
df_player_time['dire_adv_lh'] = df_player_time.apply(lambda x: x['dire_lh'] - x['radiant_lh'], axis=1)

In [11]:
df_player_time.head()

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,radiant_xp,dire_gold,dire_lh,dire_xp,radiant_adv_nw,radiant_adv_xp,radiant_adv_lh,dire_adv_nw,dire_adv_xp,dire_adv_lh
1,0,60,409,0,63,142,1,186,168,0,...,692,1370,10,692,-257,0,-8,257,-421,8
2,0,120,546,0,283,622,4,645,330,0,...,2376,2726,26,2318,-255,58,-16,255,-153,16
3,0,180,683,1,314,927,9,1202,430,0,...,3634,4057,40,3543,-567,91,-17,567,53,17
4,0,240,956,1,485,1264,11,1583,530,0,...,5285,5500,53,5007,-550,278,-20,550,57,20
5,0,300,1056,1,649,1451,13,1810,630,0,...,6484,6984,71,6574,-1037,-90,-25,1037,627,25


In [12]:
# Merge match results with match flow
final_df = pd.merge(df_player_time, df_match, left_on='match_id', right_on='match_id')

#Get match status @15minutes to predict Win%
final_df = final_df.loc[final_df.times==900]

final_df.head()

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,dire_adv_lh,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win
14,0,900,3324,5,2914,5444,64,6117,2869,10,...,39,1446750112,2375,1982,4,3,63,1,22,True
56,1,900,3184,16,3307,7849,110,8835,4041,15,...,-9,1446753078,2582,0,1846,63,0,221,22,False
101,2,900,3748,27,3855,4542,42,5571,5438,37,...,-36,1446764586,2716,256,1972,63,48,190,22,False
148,3,900,3687,10,4310,5598,49,4127,3915,26,...,7,1446765723,3085,4,1924,51,3,40,22,False
201,4,900,7099,112,6938,7001,64,6732,3973,50,...,-150,1446796385,1887,2047,0,0,63,58,22,True


In [13]:
final_df = final_df.drop_duplicates()

In [14]:
final_df.columns

Index(['match_id', 'times', 'gold_t_0', 'lh_t_0', 'xp_t_0', 'gold_t_1',
       'lh_t_1', 'xp_t_1', 'gold_t_2', 'lh_t_2', 'xp_t_2', 'gold_t_3',
       'lh_t_3', 'xp_t_3', 'gold_t_4', 'lh_t_4', 'xp_t_4', 'gold_t_128',
       'lh_t_128', 'xp_t_128', 'gold_t_129', 'lh_t_129', 'xp_t_129',
       'gold_t_130', 'lh_t_130', 'xp_t_130', 'gold_t_131', 'lh_t_131',
       'xp_t_131', 'gold_t_132', 'lh_t_132', 'xp_t_132', 'radiant_gold',
       'radiant_lh', 'radiant_xp', 'dire_gold', 'dire_lh', 'dire_xp',
       'radiant_adv_nw', 'radiant_adv_xp', 'radiant_adv_lh', 'dire_adv_nw',
       'dire_adv_xp', 'dire_adv_lh', 'start_time', 'duration',
       'tower_status_radiant', 'tower_status_dire', 'barracks_status_dire',
       'barracks_status_radiant', 'first_blood_time', 'game_mode',
       'radiant_win'],
      dtype='object')

In [15]:
# add dire_win
final_df['dire_win'] = final_df['radiant_win'].map({'True': 'False', 'False': 'True'})

# Use advantages as features
final_df = final_df.filter(['match_id','radiant_adv_nw', 'radiant_adv_xp', 'radiant_adv_lh','radiant_win'])
final_df.head()

,match_id,radiant_adv_nw,radiant_adv_xp,radiant_adv_lh,radiant_win
14,0,707,1238,-39,True
56,1,-193,-1224,9,False
101,2,1035,1470,36,False
148,3,2464,2822,-7,False
201,4,6967,5261,150,True


In [16]:
final_df = final_df.drop_duplicates().set_index('match_id')
final_df.head()

,radiant_adv_nw,radiant_adv_xp,radiant_adv_lh,radiant_win
match_id,,,,
0,707,1238,-39,True
1,-193,-1224,9,False
2,1035,1470,36,False
3,2464,2822,-7,False
4,6967,5261,150,True


In [17]:
final_df.to_csv('data.csv')